# Receiving
## Example 3. Pub/Sub with fanout exchange.

### Temporary queues
As you may remember previously we were using queues which had a specified name (remember hello and task_queue?). Being able to name a queue was crucial for us -- we needed to point the workers to the same queue. Giving a queue a name is important when you want to share the queue between producers and consumers.


In [13]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='192.168.99.100'))
channel = connection.channel()

channel.exchange_declare(exchange='logs',
                         type='fanout')

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>


But that's not the case for our logger. **We want to hear about all log messages, not just a subset of them. We're also interested only in currently flowing messages not in the old ones. To solve that we need two things.**

**Firstly**, whenever we connect to Rabbit we need a fresh, empty queue. To do it we could create a queue with a random name, or, even better - let the server choose a random queue name for us. We can do this by not supplying the queue parameter to queue_declare:




In [14]:
# result = channel.queue_declare()

At this point result.method.queue contains a random queue name. For example it may look like amq.gen-JzTY20BRgKO-HjmUJj0wLg.

**Secondly, once we disconnect the consumer the queue should be deleted. There's an exclusive flag for that:**



In [15]:
result = channel.queue_declare(exclusive=True)
queue_name = result.method.queue

### Bindings
![](https://www.rabbitmq.com/img/tutorials/bindings.png)

We've already created a fanout exchange and a queue. Now we need to tell the exchange to send messages to our queue. That relationship between exchange and a queue is called a binding.



In [16]:
channel.queue_bind(exchange='logs',
                   queue=queue_name)

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

From now on the logs exchange will append messages to our queue.



In [17]:
print ' [*] Waiting for logs. To exit press CTRL+C'

def callback(ch, method, properties, body):
    print " [x] %r" % (body,)

channel.basic_consume(callback,
                      queue=queue_name,
                      no_ack=True)

channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x] 'info: Hello World!'
 [x] 'info: Hello World!'
 [x] 'info: Hello World!'


KeyboardInterrupt: 

In [18]:
connection.close()
